# Project: Planning my next holidays : Get coordinates, temperature and plot ☀️

Let's use APIs and web-scraping to determine what would be the nicest places to visit for a last-minute holiday next week!

According to <a href=https://one-week-in.com/35-cities-to-visit-in-france/ target=_blank>this website</a>, here are the 35 best places to visit in France in 2020:

* Mont Saint Michel, St Malo, Bayeux, Le Havre, Rouen, Paris, Amiens,
* Lille, Strasbourg, Chateau du Haut Koenigsbourg, Colmar, Eguisheim,
* Besancon, Dijon, Annecy, Grenoble, Lyon, Gorges du Verdon,
* Bormes les Mimosas, Cassis, Marseille, Aix en Provence, Avignon,
* Uzes, Nimes, Aigues Mortes, Saintes Maries de la mer, Collioure,
* Carcassonne, Ariege, Toulouse, Montauban, Biarritz, Bayonne, La Rochelle


In [13]:
#!pip install requests
#!pip install pandas
#!pip install plotly==5.9.0

In [14]:
import requests
import pandas as pd
import plotly.express as px
import os
from datetime import date
import datetime

In [15]:
city_names = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", 
    "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes",
    "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Foix",
    "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

In [16]:
columns = ['city_id' , 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature']
dataset = pd.DataFrame(columns=columns)

In [17]:
LINK_OPEN_STREET= 'https://nominatim.openstreetmap.org/search'

In [18]:
LINK_OPEN_WEATHER = "https://api.openweathermap.org/data/3.0/onecall"

In [19]:
API_KEY_OPENWEATHER = os.getenv('API_KEY_OPENWEATHER')

In [20]:
for i in range(len(city_names)):
    
    print("Making requests for {}".format(city_names[i]))
    
    # Use nominatim api to get GPS coordinates of city
    par = {
        "city": city_names[i],
        "country": "France",
        "format": "json"
    }
    #1. Get the gps coordinates of all the cities
    r = requests.get(LINK_OPEN_STREET, params=par)
    res = r.json()
    
    dataset.loc[i,'city_id'] = i
    dataset.loc[i,'name'] = city_names[i]
    dataset.loc[i, 'latitude'] = res[0]['lat']
    dataset.loc[i, 'longitude'] = res[0]['lon']
    
    # Use openweathermap api to get weather for the 7 next days
    par = {
        "lat": dataset.loc[i, 'latitude'],
        "lon": dataset.loc[i, 'longitude'],
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": API_KEY_OPENWEATHER,
    }
    #Get some information about the weather for the cities and put it in a DataFrame.
    r = requests.get(LINK_OPEN_WEATHER, params=par)
    res = r.json()
    
    # Compute expected volume of rain
    expected_rain = 0
    #print(res)
    for d in res['daily']:
        if 'rain' in d.keys():
            expected_rain += d['pop']*d['rain']
            
    # Compute average day temperature
    temperatures = pd.Series([d['temp']['day'] for d in res['daily']])
    mean_temperature = temperatures.mean()

    # Extract most probable weather
    weathers = pd.Series([d['weather'][0]['main'] for d in res['daily']])
    main_weather = weathers.mode()[0]
    
    dataset.loc[i, 'main_weather'] = main_weather
    dataset.loc[i,'expected_rain'] = expected_rain
    dataset.loc[i,'day_temperature'] = mean_temperature
    

Making requests for Mont Saint Michel
Making requests for St Malo
Making requests for Bayeux


Making requests for Le Havre
Making requests for Rouen
Making requests for Paris
Making requests for Amiens
Making requests for Lille
Making requests for Strasbourg
Making requests for Chateau du Haut Koenigsbourg
Making requests for Colmar
Making requests for Eguisheim
Making requests for Besancon
Making requests for Dijon
Making requests for Annecy
Making requests for Grenoble
Making requests for Lyon
Making requests for Gorges du Verdon
Making requests for Bormes les Mimosas
Making requests for Cassis
Making requests for Marseille
Making requests for Aix en Provence
Making requests for Avignon
Making requests for Uzes
Making requests for Nimes
Making requests for Aigues Mortes
Making requests for Saintes Maries de la mer
Making requests for Collioure
Making requests for Carcassonne
Making requests for Foix
Making requests for Toulouse
Making requests for Montauban
Making requests for Biarritz
Making requests for Bayonne
Making requests for La Rochelle


In [21]:
#3. Determine the list of cities where the weather will be the nicest within the next 7 days.
dataset.loc[:,'rank'] = dataset['expected_rain'].rank(method='min')
dataset.loc[:,'inverted_rank'] = dataset['expected_rain'].rank(method='min', ascending=False)

dataset = dataset.sort_values(by=['expected_rain', 'day_temperature'], ascending = [True, False]).reset_index(drop=True)
display(dataset)

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,30,Toulouse,43.6044622,1.4442469,Rain,6.3711,16.17125,1.0,35.0
1,27,Collioure,42.52505,3.0831554,Clouds,6.4285,18.3,2.0,34.0
2,28,Carcassonne,43.2130358,2.3491069,Rain,9.9008,16.54,3.0,33.0
3,33,Bayonne,43.4945144,-1.4736657,Rain,10.52,17.745,4.0,32.0
4,8,Strasbourg,48.584614,7.7507127,Clouds,11.2146,14.60125,5.0,31.0
5,32,Biarritz,43.4832523,-1.5592776,Rain,11.44,16.86375,6.0,30.0
6,1,St Malo,48.649518,-2.0260409,Rain,14.64,13.28375,7.0,29.0
7,7,Lille,50.6365654,3.0635282,Rain,17.0505,13.81125,8.0,28.0
8,0,Mont Saint Michel,48.6359541,-1.511459954959514,Rain,17.1612,13.7375,9.0,27.0
9,31,Montauban,44.0175835,1.3549991,Clouds,18.155,16.21875,10.0,26.0


In [22]:
print('Best places for a trip next week are : ')

for i,row in dataset.loc[dataset['rank']==1,:].iterrows():
    print("{} -- Mostly {} with temperature {} °C".format(row['name'], row['main_weather'], row['day_temperature']))

Best places for a trip next week are : 
Toulouse -- Mostly Rain with temperature 16.17125 °C


In [23]:
dataset.loc[:,'latitude'] = dataset['latitude'].astype('float')
dataset.loc[:,'longitude'] = dataset['longitude'].astype('float')
dataset.loc[:,'expected_rain'] = dataset['expected_rain'].astype('float')
dataset.loc[:,'day_temperature'] = dataset['day_temperature'].astype('float')
dataset.loc[:,'rank'] = dataset['rank'].astype('int')
dataset.loc[:,'inverted_rank'] = dataset['inverted_rank'].astype('int')

In [24]:
#4. Save all the results in a .csv file, you will use it later
if not os.path.isdir("res"):
    os.mkdir("res")
dataset.to_csv('res/cities.csv', index=False)

In [25]:
import plotly
print(plotly.__version__)

5.9.0


In [26]:
#5. Use plotly to display the best destinations on a map.
fig = px.scatter_mapbox(dataset, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['main_weather', 'expected_rain', 'day_temperature'], 
                        color = 'day_temperature', color_continuous_scale = 'Bluered', size = 'inverted_rank',
                        mapbox_style="carto-positron")
fig.show()

# Project: Scrape hotels for each cities for planning my next holidays ☀️

Let's create a script that allows to get some information about all the hotels in a given city on <a href="https://www.booking.com" target="_blank">www.booking.com</a> 🧙

**We strongly recommend that you use Scrapy, it will be much easier!**

You can scrap as many information as you want, but we suggest that you get at least:
* The hotel name, 
* The url to its booking.com page, 
* Its coordinates: latitude and longitude,
* The score given by the website users,
* The text description of the hotel.

In [27]:
from bs4 import BeautifulSoup
import json

In [28]:
HEADERS = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"
}

In [29]:
def get_hotels_by_city(city):
    today = date.today()
    week = today + datetime.timedelta(days=7)
    start_day = today.strftime("%Y-%m-%d")
    end_day = week.strftime("%Y-%m-%d")
    URL=f"https://www.booking.com/searchresults.fr.html?ss={city.replace(' ', '+')}&lang=fr&src=city&checkin={start_day}&checkout={end_day}"
    print(URL)
    page = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(page.content, 'html.parser')
    hotels= []
    items = soup.find_all("div", {"data-testid" : "property-card"})
    for item in items:
        score= item.find("div", {"class": "a3b8729ab1 d86cee9b25"})
        if score is not None:
            score = score.text.split("Avec une note de")[0]
            score = float(score.replace(",", "."))
        hotel = {
            "name": item.find_all("div", {"data-testid": "title"})[0].text.strip(),
            "url": item.find_all("a")[0]["href"],
            "score": score,
        }
        result_desc = requests.get(hotel["url"], headers=HEADERS)
        des_bs4 = BeautifulSoup(result_desc.content, 'html.parser')
        hotel["description"] = des_bs4.find("p", {"data-testid": "property-description"}).text.replace("\n", "")
        extrat_values = des_bs4.find_all("div", {"class": "a53cbfa6de ebbf62ced0"})
        hotel["extrat"] = list(map(lambda item: item.text, extrat_values))
        hotel["address"] = des_bs4.find("span", {"data-source": "top_link"}).text.replace("\n", "")
        coords = des_bs4.find("a",  {"id":"hotel_header"})["data-atlas-latlng"].split(",")
        hotel["latitude"] = float(coords[0])
        hotel["longitude"] = float(coords[1])
        hotels.append(hotel)
    return hotels

def process_extract_hotels_by_city(city):
    #hear to create folder
    if not os.path.isdir("res"):
        os.mkdir("res")
    hotels = get_hotels_by_city(city)
    json_object = json.dumps(hotels, indent=2)
    with open(f"res/hotels_{city.replace(' ', '-')}.json", "w") as outfile:
        outfile.write(json_object)

def process_cities(cities):
    for city in cities:
        print(f"Processing to scrape {city}")
        process_extract_hotels_by_city(city)

In [30]:
process_cities(city_names)

Processing to scrape Mont Saint Michel
https://www.booking.com/searchresults.fr.html?ss=Mont+Saint+Michel&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07


Processing to scrape St Malo
https://www.booking.com/searchresults.fr.html?ss=St+Malo&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Bayeux
https://www.booking.com/searchresults.fr.html?ss=Bayeux&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Le Havre
https://www.booking.com/searchresults.fr.html?ss=Le+Havre&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Rouen
https://www.booking.com/searchresults.fr.html?ss=Rouen&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Paris
https://www.booking.com/searchresults.fr.html?ss=Paris&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Amiens
https://www.booking.com/searchresults.fr.html?ss=Amiens&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape Lille
https://www.booking.com/searchresults.fr.html?ss=Lille&lang=fr&src=city&checkin=2024-03-31&checkout=2024-04-07
Processing to scrape

# Project: Planning my next holidays ☀️

Today, you'll clean and analyze the data that you collected about cities and hotels. In the end, you will be able to save it in a S3 bucket in your AWS account.

Import all libraries you need at first:

In [31]:
#!pip install boto3

In [32]:
import glob
import plotly.express as px
import boto3

In [33]:
print(boto3.__version__)

1.24.28


1. Read the `.csv` file that contains information about cities and weather

In [34]:
cities = pd.read_csv('res/cities.csv')
cities.head()

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,30,Toulouse,43.604462,1.444247,Rain,6.3711,16.17125,1,35
1,27,Collioure,42.525050,3.083155,Clouds,6.4285,18.30000,2,34
2,28,Carcassonne,43.213036,2.349107,Rain,9.9008,16.54000,3,33
3,33,Bayonne,43.494514,-1.473666,Rain,10.5200,17.74500,4,32
4,8,Strasbourg,48.584614,7.750713,Clouds,11.2146,14.60125,5,31


2. Read the `.json` files containing information about hotels and save it into a single pandas Dataframe. You can save as much information as you want, but don't forget to save at least these important ones:

- name of the city
- id of the city (you will find it in the `.csv` file about cities)
- create a column containing a unique identifier of the hotel (hotel_id)
- name of the hotel

👀 _If you store textual information, make sure that you clean it such that it is readable._

In [35]:
# Get all files in `res` folder starting by `2_hotels_` and 
# finishing by `.json`
hotel_files = glob.glob('res/hotels_*.json')

# Create a new DataFrame
hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id', 'name', 'url', 'latitude', 'longitude', 'score', 'description'])
print(hotel_files)
# Iterate over all JSON files
for f in hotel_files:
    city_name = f.split('_')[1].split('.')[0].replace("-"," ")
    city_id = cities.loc[cities['name'] == city_name,'city_id'].values[0]
    
    print("Processing {}".format(city_name))
    
    # Read json files and add hotel_id, city_id and city_name into DataFrames
    temp_dataset = pd.read_json(f)
    if temp_dataset.empty:
        print(f"file {f} is empty")
        continue
    temp_dataset = temp_dataset.reset_index().rename({'index': 'hotel_id'}, axis = 1)
    temp_dataset.loc[:,'city_id'] = city_id
    temp_dataset.loc[:,'city_name'] = city_name
    
    # Clean text fields
    temp_dataset.loc[:, 'name'] = temp_dataset['name'].str.replace('\n', '')
    temp_dataset.loc[:, 'url'] = temp_dataset['url'].str.replace('\n', '')
    temp_dataset.loc[:, 'description'] = temp_dataset['description'].str.replace('\n', '')
    
    # Append to hotels dataframe
    hotels = pd.concat([hotels, temp_dataset])

['res\\hotels_Aigues-Mortes.json', 'res\\hotels_Aix-en-Provence.json', 'res\\hotels_Amiens.json', 'res\\hotels_Annecy.json', 'res\\hotels_Avignon.json', 'res\\hotels_Bayeux.json', 'res\\hotels_Bayonne.json', 'res\\hotels_Besancon.json', 'res\\hotels_Biarritz.json', 'res\\hotels_Bormes-les-Mimosas.json', 'res\\hotels_Carcassonne.json', 'res\\hotels_Cassis.json', 'res\\hotels_Chateau-du-Haut-Koenigsbourg.json', 'res\\hotels_Collioure.json', 'res\\hotels_Colmar.json', 'res\\hotels_Dijon.json', 'res\\hotels_Eguisheim.json', 'res\\hotels_Foix.json', 'res\\hotels_Gorges-du-Verdon.json', 'res\\hotels_Grenoble.json', 'res\\hotels_La-Rochelle.json', 'res\\hotels_Le-Havre.json', 'res\\hotels_Lille.json', 'res\\hotels_Lyon.json', 'res\\hotels_Marseille.json', 'res\\hotels_Mont-Saint-Michel.json', 'res\\hotels_Montauban.json', 'res\\hotels_Nimes.json', 'res\\hotels_Paris.json', 'res\\hotels_Rouen.json', 'res\\hotels_Saintes-Maries-de-la-mer.json', 'res\\hotels_St-Malo.json', 'res\\hotels_Strasbour

In [36]:
# Convert columns to convenient types
hotels.loc[:, 'city_id'] = hotels['city_id'].astype('int')
hotels.loc[:, 'hotel_id'] = hotels['hotel_id'].astype('int')
hotels.loc[:, 'latitude'] = hotels['latitude'].astype('float')
hotels.loc[:, 'longitude'] = hotels['longitude'].astype('float')
hotels.loc[:, 'score'] = hotels['score'].astype('float')

In [37]:
# Sanity check
hotels.head()

,city_id,city_name,hotel_id,name,url,latitude,longitude,score,description,extrat,address
0,25,Aigues Mortes,0,Hôtel Saint Louis,https://www.booking.com/hotel/fr/saint-louis-a...,43.567220,4.189344,8.4,"Situé dans le cœur historique d'Aigues-Mortes,...",[],"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F..."
1,25,Aigues Mortes,1,Hotel Canal Aigues Mortes,https://www.booking.com/hotel/fr/canal-aigues-...,43.572988,4.194825,8.5,L’hôtel Canal Aigues Mortes vous accueille sur...,[],"440 Route De Nimes, 30220 Aigues-Mortes, France"
2,25,Aigues Mortes,2,La Villa Mazarin,https://www.booking.com/hotel/fr/la-villa-maza...,43.564987,4.191752,9.2,La Villa Mazarin est construite dans un bâtime...,[],"35 boulevard Gambetta, 30220 Aigues-Mortes, Fr..."
3,25,Aigues Mortes,3,Maison des Croisades,https://www.booking.com/hotel/fr/des-croisades...,43.568997,4.188298,8.8,"Située à Aigues-Mortes, la Maison des Croisade...",[],"2 RUE DU PORT, 30220 Aigues-Mortes, France"
4,25,Aigues Mortes,4,Hôtel Le Médiéval,https://www.booking.com/hotel/fr/le-medieval.f...,43.571866,4.193662,8.6,L'Hôtel Le Médiéval est situé sur les rives du...,[],"221 Avenue Du Pont De Provence, 30220 Aigues-M..."


In [38]:
# Save hotels DataFrame into .csv file
hotels.to_csv('res/hotels.csv', index=False)

In [39]:
hotels_with_score = hotels.loc[hotels['score'].notnull(),:]

fig = px.scatter_mapbox(hotels_with_score, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['description'],
                        color = 'score', color_continuous_scale = 'thermal',
                        mapbox_style="carto-positron")
fig.show()

In [40]:
YOUR_ACCESS_KEY_ID = os.getenv("YOUR_ACCESS_KEY_ID")
YOUR_SECRET_ACCESS_KEY = os.getenv ("YOUR_SECRET_ACCESS_KEY")

5. Use `boto3` to save the DataFrames about cities and hotels into `.csv` files located in a new S3 bucket in your AWS account:

In [42]:
BUCKET_NAME = "my-project-kayak-dustin"

configuration={
        'LocationConstraint': 'eu-central-1'
}

# Initiate a new session
session = boto3.Session(aws_access_key_id=YOUR_ACCESS_KEY_ID, 
                        aws_secret_access_key=YOUR_SECRET_ACCESS_KEY)

# Declare s3 object and create a new bucket
s3 = session.resource("s3")

bucket = s3.Bucket(BUCKET_NAME)
if bucket.creation_date:
    print(f"Bucket already created {bucket.creation_date}")
else:
    print("Creating the bucket")
    bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration=configuration)

# Export hotels to CSV file and upload it
hotels_csv = hotels.to_csv()
put_object = bucket.put_object(Key = "hotels.csv", Body = hotels_csv)
print("Finish to upload the file hotels.csv")

# Do the same for cities
cities_csv = cities.to_csv()
put_object = bucket.put_object(Key = "cities.csv", Body = cities_csv)
print("Finish to upload the file cities.csv")

Bucket already created 2024-03-29 21:40:55+00:00


Finish to upload the file hotels.csv
Finish to upload the file cities.csv


# Project: planning my next holidays ☀️

Let's put your dataset in a postgreSQL database on Amazon RDS ! Then, you can use the SQL syntax to make requests and decide where you'll go for your holidays. 😎


In [43]:
!pip install psycopg2-binary

In [44]:
from sqlalchemy import create_engine

In [46]:
cities = pd.read_csv('res/cities.csv')
hotels = pd.read_csv('res/hotels.csv')

In [47]:
cities.head()

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,30,Toulouse,43.604462,1.444247,Rain,6.3711,16.17125,1,35
1,27,Collioure,42.525050,3.083155,Clouds,6.4285,18.30000,2,34
2,28,Carcassonne,43.213036,2.349107,Rain,9.9008,16.54000,3,33
3,33,Bayonne,43.494514,-1.473666,Rain,10.5200,17.74500,4,32
4,8,Strasbourg,48.584614,7.750713,Clouds,11.2146,14.60125,5,31


In [48]:
hotels.head()

,city_id,city_name,hotel_id,name,url,latitude,longitude,score,description,extrat,address
0,25,Aigues Mortes,0,Hôtel Saint Louis,https://www.booking.com/hotel/fr/saint-louis-a...,43.567220,4.189344,8.4,"Situé dans le cœur historique d'Aigues-Mortes,...",[],"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F..."
1,25,Aigues Mortes,1,Hotel Canal Aigues Mortes,https://www.booking.com/hotel/fr/canal-aigues-...,43.572988,4.194825,8.5,L’hôtel Canal Aigues Mortes vous accueille sur...,[],"440 Route De Nimes, 30220 Aigues-Mortes, France"
2,25,Aigues Mortes,2,La Villa Mazarin,https://www.booking.com/hotel/fr/la-villa-maza...,43.564987,4.191752,9.2,La Villa Mazarin est construite dans un bâtime...,[],"35 boulevard Gambetta, 30220 Aigues-Mortes, Fr..."
3,25,Aigues Mortes,3,Maison des Croisades,https://www.booking.com/hotel/fr/des-croisades...,43.568997,4.188298,8.8,"Située à Aigues-Mortes, la Maison des Croisade...",[],"2 RUE DU PORT, 30220 Aigues-Mortes, France"
4,25,Aigues Mortes,4,Hôtel Le Médiéval,https://www.booking.com/hotel/fr/le-medieval.f...,43.571866,4.193662,8.6,L'Hôtel Le Médiéval est situé sur les rives du...,[],"221 Avenue Du Pont De Provence, 30220 Aigues-M..."


2. Create and configure a RDS instance in your AWS account. 

3. Use SQLAlchemy to create an engine that is connected to the remote database:

In [50]:
YOUR_PASSWORD = "tfax27nk"
YOUR_USERNAME = "tfax27nk"
YOUR_HOST = "localhost:5432"
table_name = "kayak"

In [51]:
# Change YOUR_USERNAME, YOUR_PASSWORD and YOUR_HOST with you credentials
engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOST}/{table_name}", echo=True)

4. Create two tables in your remote database : `cities` and `hotels`, each one containing the information from the different `.csv` files you saved last time.


In [52]:
try:
    cities.to_sql("cities", engine)
except:
    print("table exists create table")

try:
    hotels.to_sql("hotels", engine)
except:
    print("table exists create table")

table exists create table
table exists create table


In [53]:
hotels.to_sql("hotels", engine)

OperationalError: (psycopg2.OperationalError) 
(Background on this error at: https://sqlalche.me/e/20/e3q8)

5. Optional: use PGAdmin to explore the database

6. Use SQL syntax to make a request that will help you get the list of the 30 best places to go (among all cities and hotels). 😎

In [ ]:
conn = engine.connect()

statement = "SELECT city_name, main_weather, day_temperature, hotels.name, description, score, url \
             FROM cities \
             INNER JOIN hotels \
             ON cities.city_id = hotels.city_id \
             WHERE rank=1 AND score IS NOT NULL \
             ORDER BY score DESC \
             LIMIT 30"

best_places = pd.read_sql(statement, conn)

In [ ]:
display(best_places)
